## Applying the RF model to predict terminal category membership

- Created by: Grace Patlewicz
- Date: 10th August 2024
This notebook outlines how to apply the RF developed to make predictions of terminal category membership.

Note: Whilst most notebooks relied upon python 3.9. The conda environment for the the model development relied upon python 3.10, pandas 1.3.4, rdkit 2023.9.5 and scikit-learn 1.4.2

In [1]:
import pickle

In [4]:
import numpy as np
import pandas as pd

from rdkit import Chem

from rdkit.Chem import AllChem

import os


In [5]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
processed_dir = TOP + 'data/processed/'
figures_dir = TOP + 'reports/figures/'

In [6]:
from sklearn.model_selection import cross_validate,KFold,ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import balanced_accuracy_score, f1_score, make_scorer, roc_auc_score
from sklearn.metrics import auc, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display = 'diagram')

In [17]:
models_dir = TOP + 'models/'

In [18]:
models_dir

'/home/grace/Documents/python/nts_pfas/models/'

In [19]:
# load the model from disk

filename = 'final_model_v2.sav'
loaded_model = pickle.load(open(models_dir+filename, 'rb'))

In [20]:
loaded_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['category']),
                                                 ('num', StandardScaler(),
                                                  ['mrgn_0', 'mrgn_1', 'mrgn_2',
                                                   'mrgn_3', 'mrgn_4', 'mrgn_5',
                                                   'mrgn_6', 'mrgn_7', 'mrgn_8',
                                                   'mrgn_9', 'mrgn_10',
                                                   'mrgn_11', 'mrgn_12',
                                                   'mrgn_13', 'mrgn_14',
                                                   'mrgn_15', 'mrgn_16',
                                                   'mrgn_17', 'mrgn_18',
                                                   'mrgn_19', 'mrgn_20',
                                                   'mrgn_21', 'mrgn_22',
                                                   'mrgn_23', 'mrgn_24',
                                                   'mrgn_25', 'mrgn_26',
                                                   'mrgn_27', 'mrgn_28',
                                                   'mrgn_29', ...])])),
                ('clf',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        max_samples=0.66, min_samples_leaf=2,
                                        n_estimators=400, random_state=42))])

In [21]:
loaded_model[1].feature_importances_

array([0.05981849, 0.00039647, 0.00752954, ..., 0.0007338 , 0.00277207,
       0.02375698])

In [22]:
import sys

In [23]:
LIB = TOP+'src/models/'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)


In [26]:
from model_functions import *

In [27]:
df = pd.read_excel(interim_dir+'final_revised_universe_wmappingdict_130524.xlsx', index_col = [0])

In [28]:
df.shape

(15525, 148)

In [29]:
data = mk_fp(df)

In [30]:
df['group_str']=[str(e) for e in df['group'] ]

In [31]:
df1 = df.set_index('dtxsid')
y1 = df1[['group_str', 'category', 'chain_length']]

In [32]:
df2 = pd.concat([data, y1], axis = 1)

In [33]:
#df2 = df2.sample(frac = 1)

In [34]:
df2

,mrgn_0,mrgn_1,mrgn_2,mrgn_3,mrgn_4,mrgn_5,mrgn_6,mrgn_7,mrgn_8,mrgn_9,...,mrgn_1017,mrgn_1018,mrgn_1019,mrgn_1020,mrgn_1021,mrgn_1022,mrgn_1023,group_str,category,chain_length
IROQAHVXXUQBOS-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,"('Aromatic PFASs', 'gte7', nan, nan)",Aromatic PFASs,8
DTXSID90897582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"('Aromatic PFASs', 'gte7', nan, nan)",Aromatic PFASs,9
DTXSID90896257,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,"('Aromatic PFASs', 'gte7', nan, nan)",Aromatic PFASs,8
DTXSID90896196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,"('Aromatic PFASs', 'gte7', nan, nan)",Aromatic PFASs,8
DTXSID90896095,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"('Aromatic PFASs', 'gte7', nan, nan)",Aromatic PFASs,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIQDQEQBQRGFDF-UHFFFAOYSA-N,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,"('unclassified', 'lt7', nan, nan)",unclassified,4
ZKYPCYQMVQMYAB-UHFFFAOYSA-N,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,"('unclassified', 'lt7', nan, nan)",unclassified,4
ZOJATUBFKQLTCD-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,"('unclassified', 'lt7', nan, nan)",unclassified,4
ZRONJOSZRXLGCA-UHFFFAOYSA-N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"('unclassified', 'lt7', nan, nan)",unclassified,4


In [35]:
from collections import Counter

In [36]:
df2['group_str'].value_counts()

group_str
('Aromatic PFASs', 'lt7', 2.0, 5.0)            1238
('PASF-based substances', 'gte7', nan, nan)     770
('PASF-based substances', 'lt7', nan, nan)      697
('Aromatic PFASs', 'lt7', 2.0, 2.0)             592
('Aromatic PFASs', 'gte7', nan, nan)            584
                                               ... 
('PFAAs, cyclic', 'lt7', 3.0, 1.0)                4
('Other PFASs, cyclic', 'gte7', 1.0, nan)         2
('unclassified', 'lt7', 2.0, 2.0)                 2
('Other PFASs, cyclic', 'gte7', 3.0, nan)         2
('PFAAs, cyclic', 'gte7', nan, nan)               1
Name: count, Length: 128, dtype: int64

In [37]:
counts = Counter([molecule for molecule in df2['group_str']])

In [38]:
len( [group for group in counts if counts[group]>=10])


117

In [39]:
small_groups = [group for group in counts if counts[group]<10]

In [40]:
len(small_groups)

11

In [41]:
small_groups

["('Other PFASs, cyclic', 'gte7', 1.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 2.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 3.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 4.0, nan)",
 "('Other PFASs, cyclic', 'lt7', 1.0, 2.0)",
 "('PFAAs, cyclic', 'gte7', nan, nan)",
 "('PFAAs, cyclic', 'lt7', 3.0, 1.0)",
 "('PFAAs, cyclic', 'lt7', 3.0, 2.0)",
 "('unclassified', 'lt7', 1.0, 1.0)",
 "('unclassified', 'lt7', 2.0, 1.0)",
 "('unclassified', 'lt7', 2.0, 2.0)"]

In [42]:
df2['group'] = df2['group_str'].apply(lambda x: 'misc_category' if x in small_groups else x)

In [43]:
cats = set(df2['group'].sort_values().tolist())

In [44]:
my_dict = {}
sorted_cats = sorted(cats)  # Sort the categories

for i, e in enumerate(sorted_cats, 1):
    a = f'Category{i}'
    my_dict[e] = a
    

In [45]:
df2['terminal_category'] = df2['group'].replace(my_dict)

In [46]:
df2.terminal_category.value_counts()

terminal_category
Category7      1238
Category30      770
Category31      697
Category4       592
Category1       584
               ... 
Category82       12
Category81       11
Category115      11
Category102      10
Category104      10
Name: count, Length: 118, dtype: int64

In [47]:
y = df2['terminal_category']

In [48]:
X = df2.drop(['group_str', 'group', 'terminal_category'], axis = 1)

In [49]:
X['category'] = X['category'].astype('category')

In [50]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [51]:
loaded_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OrdinalEncoder(encoded_missing_value=-1,
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['category']),
                                                 ('num', StandardScaler(),
                                                  ['mrgn_0', 'mrgn_1', 'mrgn_2',
                                                   'mrgn_3', 'mrgn_4', 'mrgn_5',
                                                   'mrgn_6', 'mrgn_7', 'mrgn_8',
                                                   'mrgn_9', 'mrgn_10',
                                                   'mrgn_11', 'mrgn_12',
                                                   'mrgn_13', 'mrgn_14',
                                                   'mrgn_15', 'mrgn_16',
                                                   'mrgn_17', 'mrgn_18',
                                                   'mrgn_19', 'mrgn_20',
                                                   'mrgn_21', 'mrgn_22',
                                                   'mrgn_23', 'mrgn_24',
                                                   'mrgn_25', 'mrgn_26',
                                                   'mrgn_27', 'mrgn_28',
                                                   'mrgn_29', ...])])),
                ('clf',
                 RandomForestClassifier(class_weight='balanced_subsample',
                                        max_samples=0.66, min_samples_leaf=2,
                                        n_estimators=400, random_state=42))])

In [52]:
features = list(X.columns)

In [53]:
feature_importances1 = pd.DataFrame({'feature': features, 'importance': loaded_model[1].feature_importances_})
feature_importances1.head()


,feature,importance
0,mrgn_0,0.059818
1,mrgn_1,0.000396
2,mrgn_2,0.007530
3,mrgn_3,0.000474
4,mrgn_4,0.000256


In [54]:
feature_importances1[feature_importances1['feature']== 'category']

,feature,importance
1024,category,0.002772


In [55]:
loaded_model[1]

RandomForestClassifier(class_weight='balanced_subsample', max_samples=0.66,
                       min_samples_leaf=2, n_estimators=400, random_state=42)

In [56]:
preds = loaded_model.predict(X)

In [57]:
preds

array(['Category1', 'Category1', 'Category1', ..., 'Category117',
       'Category117', 'Category117'], dtype=object)

In [58]:
pred_prob = loaded_model.predict_proba(X)

In [59]:
X.shape

(15525, 1026)

In [60]:
len(preds)

15525

In [61]:
len(pred_prob)

15525

In [62]:
pred_cat_proba = pd.DataFrame(pred_prob, index = X.index, columns = list(my_dict.values()))

In [63]:
pred_cats = pd.DataFrame(preds, index = X.index, columns = ['predicted_category'])

In [64]:
df3 = pd.concat([df2[['category', 'chain_length', 'group', 'terminal_category']], pred_cats, pred_cat_proba], axis = 1)

In [66]:
reverse_dict = {v:k for k,v in my_dict.items()}

In [68]:
#reverse_dict

In [69]:
reverse_f= 'reverse_dict2.pkl'
pickle.dump(reverse_dict, open(external_dir+reverse_f, 'wb'))
my_dict_f = 'my_dict2.pkl'
pickle.dump(my_dict, open(external_dir+my_dict_f, 'wb'))

In [70]:
df3['predicted_terminal'] = df3['predicted_category'].replace(reverse_dict)

In [71]:
df3.head()

,category,chain_length,group,terminal_category,predicted_category,Category1,Category2,Category3,Category4,Category5,...,Category110,Category111,Category112,Category113,Category114,Category115,Category116,Category117,Category118,predicted_terminal
IROQAHVXXUQBOS-UHFFFAOYSA-N,Aromatic PFASs,8,"('Aromatic PFASs', 'gte7', nan, nan)",Category1,Category1,0.996000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,"('Aromatic PFASs', 'gte7', nan, nan)"
DTXSID90897582,Aromatic PFASs,9,"('Aromatic PFASs', 'gte7', nan, nan)",Category1,Category1,0.843482,0.000000,0.001426,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.001009,0.003474,0.000000,0.000000,0.0,"('Aromatic PFASs', 'gte7', nan, nan)"
DTXSID90896257,Aromatic PFASs,8,"('Aromatic PFASs', 'gte7', nan, nan)",Category1,Category1,0.594020,0.005263,0.007892,0.007320,0.0,...,0.000387,0.002152,0.0,0.0,0.001886,0.000000,0.003913,0.014581,0.0,"('Aromatic PFASs', 'gte7', nan, nan)"
DTXSID90896196,Aromatic PFASs,8,"('Aromatic PFASs', 'gte7', nan, nan)",Category1,Category1,0.602849,0.009611,0.001760,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000415,0.000000,0.0,"('Aromatic PFASs', 'gte7', nan, nan)"
DTXSID90896095,Aromatic PFASs,8,"('Aromatic PFASs', 'gte7', nan, nan)",Category1,Category1,0.447069,0.006399,0.000000,0.002037,0.0,...,0.011202,0.000000,0.0,0.0,0.000766,0.003603,0.000000,0.045920,0.0,"('Aromatic PFASs', 'gte7', nan, nan)"


In [72]:
X.category.unique().tolist()

['Aromatic PFASs',
 'HFCs',
 'Other PFASs',
 'Other PFASs, cyclic',
 'PASF-based substances',
 'PFAA precursors',
 'PFAA precursors, cyclic',
 'PFAAs',
 'PFAAs, cyclic',
 'PolyFCA derivatives',
 'Polyfluoroalkanes',
 'Polyfluoroalkyl acids',
 'Polyfluoroalkyl acids, cyclic',
 'Si PFASs',
 'n:2 fluorotelomer-based substances',
 'others',
 'others, cyclic',
 'unclassified']

In [73]:
small_groups

["('Other PFASs, cyclic', 'gte7', 1.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 2.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 3.0, nan)",
 "('Other PFASs, cyclic', 'gte7', 4.0, nan)",
 "('Other PFASs, cyclic', 'lt7', 1.0, 2.0)",
 "('PFAAs, cyclic', 'gte7', nan, nan)",
 "('PFAAs, cyclic', 'lt7', 3.0, 1.0)",
 "('PFAAs, cyclic', 'lt7', 3.0, 2.0)",
 "('unclassified', 'lt7', 1.0, 1.0)",
 "('unclassified', 'lt7', 2.0, 1.0)",
 "('unclassified', 'lt7', 2.0, 2.0)"]

## Procedure for making a new prediction

1. Take SMILES and apply the PFAS-Atlas code to assign a first-class and second-class assignment

2. Use table to determine whether a second-class assignment is needed


|         PFAS-Atlas first class        |          Primary category          |
|:----------------------------:|:----------------------------------:|
|            PFAAs             |               PFAAs                |
|        PFAAs, cyclic         |           PFAAs, cyclic            |
|       PFAA precursors        |          PFAA precursors           |
|       PFAA precursors        |       PASF-based substances        |
|       PFAA precursors        | n:2 fluorotelomer-based substances |
|       PFAA precursors        |                HFCs                |
|   PFAA precursors, cyclic    |      PFAA precursors, cyclic       |
|    Polyfluoroalkyl acids     |       Polyfluoroalkyl acids        |
|    Polyfluoroalkyl acids     |        PolyFCA derivatives         |
| Polyfluoroalkyl acids,cyclic |    Polyfluoroalkyl acids,cyclic    |
|          Other PFAS          |             Other PFAS             |
|          Other PFAS          |           Aromatic PFASs           |
|                              |              Si PFASs              |
|                              |         Polyfluoroalkanes          |
|                              |               others               |
|      Other PFAS,cyclic       |         Other PFAS,cyclic          |
|      Other PFAS,cyclic       |           others, cyclic           |
|           Not PFAS           |            Unclassified            |


3. Determine the chain length
- Convert SMILES into a RDKIT mol object

In [74]:
def chain_length(mol, ch=30):
    mysr = 'C(F)(F)'
    mylst = []
    for n in range(1, ch):
        a = mol.HasSubstructMatch(Chem.MolFromSmarts(''.join(mysr * n)))
        mylst.append(a)
    return mylst.index(False)


4. Use the SMILES and create a pandas df of Morgan fingerprints using a radius of 3 and bitvector length of 1024 with RDKIT

5. Create a dataframe where the index is the chemical identifier, the first set of columns correspond to the Morgan FP created in Step 4, the category column and the chain length
- Convert the category column to a 'category' type in pandas

6. Load the scikit model as a pickle file and run predictions on the df constructed. 
<br>
```preds = model.predict(df)```
<br>
```preds = model.predict_prob(df)```
- the predict function will return the most likely terminal category whereas the predict_prob will return an array of the all probabilities across all terminal categories

7. Use the my_dict and reverse_dict dictionaries to convert the model category labels back to the original terminal category names

- If df if the dataframe of substances and their inputs, create a new df of the predictions per Step 6
- Use the reverse_dict to convert the predicted category label back to the terminal category label using the reverse_dict as follows: 
```df3['predicted_terminal'] = df3['predicted_category'].replace(reverse_dict)```
- If the probabilities across categories are desired using the my_dict dictionary to transform the predicted probabilities into a df with the substance identifiers as the index and the column headings as the predicted model category labels as follows:
```pred_cat_proba = pd.DataFrame(pred_prob, index = df.index, columns = list(my_dict.values()))```

In [75]:
def mgrn_fp(dtx, s):
    mol = Chem.MolFromSmiles(s)
    mgrn_df = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(mol,3,1024))] )
    mgrn_df.columns = ['mrgn_%d'%i for i in mgrn_df.columns]
    mgrn_df.index = [dtx]
    return mgrn_df, mol

In [76]:
categories = ['Aromatic PFASs',
 'HFCs',
 'Other PFASs',
 'Other PFASs, cyclic',
 'PASF-based substances',
 'PFAA precursors',
 'PFAA precursors, cyclic',
 'PFAAs',
 'PFAAs, cyclic',
 'PolyFCA derivatives',
 'Polyfluoroalkanes',
 'Polyfluoroalkyl acids',
 'Polyfluoroalkyl acids, cyclic',
 'Si PFASs',
 'n:2 fluorotelomer-based substances',
 'others',
 'others, cyclic',
 'unclassified']

In [79]:
test = make_df( dtx='dtx', cat = 'unclassified' , s= 'CCCN(CCNC(=O)c1ccc(Cc2ccc(C(O)=O)cc2)cc1)S(=O)(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)F')

In [78]:
def make_df(s, dtx, cat='unclassified'):
    if cat not in categories:
        raise ValueError
    df1, mol = mgrn_fp(dtx,s)
    df1['category'] = cat
    df1['chain_length'] = chain_length(mol)
    df1['category']  = df1['category'].astype('category')
    return df1

In [80]:
loaded_model.predict(test)

array(['Category1'], dtype=object)

In [82]:
#reverse_dict

In [83]:
def make_prediction(df, model):
    pred = model.predict(df)
    reverse_dict[pred[0]]
    return reverse_dict[pred[0]]

In [84]:
make_prediction(test, loaded_model)

"('Aromatic PFASs', 'gte7', nan, nan)"